# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Image Preprocessing Layer, Image Augmentation Layer를 통해 이미지를 변형시켜 봅시다.
            * Transfer Learning(Inception V3)
* 성능 가이드
    * Accuracy : 0.34~0.74

## 1.환경설정

* **세부 요구사항**
    - 경로 설정 : Google Colaboratory의 가상 컴퓨터 안에 데이터셋을 다운로드 받고 압축을 해제하세요.
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.

In [1]:
!pip install keras --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.0 which is incompatible.


In [2]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

### (1) 라이브러리 불러오기

In [3]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

### (2) 데이터 불러오기

#### 1) 데이터 다운로드

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# def download_file(file_id, save_path) :
#     if os.path.exists(save_path) :
#         print(f'{save_path} 파일이 이미 존재합니다.')
#         return

#     gdown.download(id=file_id, output=save_path, quiet=False)

In [8]:
# file_id = '10DPW-bM7uNvX0v5rd_9PSxPc-CJX2XtZ'

# download_file(file_id, 'car_images.zip')

#### 2) 데이터 압축 해제

In [7]:
path = "/content/drive/MyDrive/KT_aivle/CNN/mini_4/Car_Images.zip"

In [8]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[:-4]}/')

In [9]:
dataset_extract(path)

Extracting: 100%|██████████| 607/607 [00:09<00:00, 65.19files/s]


#### 2) 데이터 조회

In [10]:
# 폴더별 이미지 데이터 갯수 확인
from glob import glob

adnormal = list(glob('/content/Car_Images/abnormal/*'))
normal = list(glob('/content/Car_Images/normal/*'))

len(normal), len(adnormal)

(302, 303)

In [11]:
from PIL import Image

In [12]:
# 정상 차량 랜덤 이미지 확인 및 형태 확인
image_path = '/content/Car_Images/abnormal/'

img_list = os.listdir(image_path) #디렉토리 내 모든 파일 불러오기
img_list_jpg = [img for img in img_list if img.endswith(".png")] #지정된 확장자만 필터링

print ("img_list_jpg: {}".format(img_list_jpg))

adnornal_np = []

for i in img_list_jpg:
    img = Image.open(image_path + i)
    img_array = np.array(img)
    adnornal_np.append(img_array)
    print(i, " 추가 완료 - 구조:", img_array.shape)

img_list_jpg: ['DALLíñE 2023-03-11 00.14.28 - a part of car without scratch.png', 'DALLíñE 2023-03-11 14.54.14 - dents of a car.png', 'DALLíñE 2023-03-11 17.27.08 - slightly dented car.png', 'DALLíñE 2023-03-11 01.25.20 - slightly scratched car.png', 'DALLíñE 2023-03-11 17.18.00 - scratched car.png', 'DALLíñE 2023-03-10 18.54.42 - slightly damaged car.png', 'DALLíñE 2023-03-11 14.50.36 - dents of a car.png', 'DALLíñE 2023-03-11 14.50.45 - dents of a car.png', 'DALLíñE 2023-03-11 18.42.57 - slightly dented car.png', 'DALLíñE 2023-03-11 01.29.53 - a little bit scratched car.png', 'DALLíñE 2023-03-11 17.27.58 - slightly dented car.png', 'DALLíñE 2023-03-11 01.22.25 - scratched car.png', 'DALLíñE 2023-03-11 18.43.34 - slightly dented car.png', 'DALLíñE 2023-03-11 14.53.25 - dents of a car.png', 'DALLíñE 2023-03-11 01.21.46 - scratched car.png', 'DALLíñE 2023-03-11 15.11.06 - dents of a car.png', 'DALLíñE 2023-03-11 15.08.00 - dents of a car.png', 'DALLíñE 2023-03-11 00.03.13 - a part of ca

In [13]:
# 파손 차량 랜덤 이미지 확인 및 형태 확인

image_path = '/content/Car_Images/normal/'

img_list = os.listdir(image_path) #디렉토리 내 모든 파일 불러오기
img_list_jpg = [img for img in img_list if img.endswith(".png")] #지정된 확장자만 필터링

print ("img_list_jpg: {}".format(img_list_jpg))

nornal_np = []

for i in img_list_jpg:
    img = Image.open(image_path + i)
    img_array = np.array(img)
    nornal_np.append(img_array)
    print(i, " 추가 완료 - 구조:", img_array.shape)

img_list_jpg: ['DALLíñE 2023-03-10 18.52.40 - photo of a part of car.png', 'DALLíñE 2023-03-11 14.25.21 - part of a car.png', 'DALLíñE 2023-03-11 14.32.34 - part of a car.png', 'DALLíñE 2023-03-10 22.22.18 - photo of a part of car.png', 'DALLíñE 2023-03-11 17.08.39 - a part of a car.png', 'DALLíñE 2023-03-11 14.14.11 - part of a car.png', 'DALLíñE 2023-03-11 14.33.23 - part of a car.png', 'DALLíñE 2023-03-11 14.42.03 - photo of part of a car.png', 'DALLíñE 2023-03-11 14.15.06 - part of a car.png', 'DALLíñE 2023-03-10 22.07.10 - photo of a part of car.png', 'DALLíñE 2023-03-11 00.51.38 - photo of a part of car.png', 'DALLíñE 2023-03-10 23.41.41 - photo of a part of car without blemish.png', 'DALLíñE 2023-03-11 01.36.44 - a part of a car.png', 'DALLíñE 2023-03-11 00.52.55 - photo of a part of car.png', 'DALLíñE 2023-03-11 14.26.52 - part of a car.png', 'DALLíñE 2023-03-10 23.41.16 - photo of a part of car without blemish.png', 'DALLíñE 2023-03-10 18.52.13 - photo of a part of car.png', '

## 2.데이터 전처리
* 모델링을 위한 데이터 구조 만들기
    * X : 이미지를 array로 변환합니다.
    * Y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.

### (1) Y : 클래스 만들기
- **세부요구사항**
    - 전체 데이터에 대한 Y를 생성합니다.
    - normal, abnormal 데이터의 수를 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [14]:
normal_y = np.zeros(302)
normal_y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [15]:
adnormal_y = np.zeros(303)+1
adnormal_y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

### (2) X : 데이터 리스트 통합
- **세부요구사항**
    - 전체 이미지 데이터를 하나의 리스트로 통합합니다.

In [16]:
img_data = np.concatenate((adnornal_np, nornal_np), axis=0)

img_data_y = np.concatenate((adnormal_y, normal_y), axis=0)


In [ ]:
print(img_data), print(len(img_data))

### (3) 데이터셋 분리
- **세부요구사항**
    - 데이터 스플릿의 비율 (1) : train set, test set = 90%, 10%으로 설정하세요.
    - 데이터 스플릿의 비율 (2) : train set, valid set = 90%, 10%으로 설정하세요.

In [17]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(img_data, img_data_y, test_size=.1, stratify=img_data_y)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=.1, stratify=train_y)


In [18]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape

((489, 1024, 1024, 3),
 (489,),
 (55, 1024, 1024, 3),
 (55,),
 (61, 1024, 1024, 3),
 (61,))

### (4) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다.
    * 이미지 파일을 불러와 Training data, Validation data, Test data 각각 array 형태로 변환해 봅시다.

## 3.모델링 I
* **세부요구사항**
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

### (1) 모델 1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

In [19]:
import keras
from keras.backend import clear_session
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, RandomTranslation, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.losses import binary_crossentropy

#### 1) 구조 설계

## 모델1

In [ ]:
clear_session()

il = Input(shape=(1024,1024,3))
al = RandomTranslation(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3))(il)
hl = Conv2D(8,kernel_size=(5,5),activation='relu')(al)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(32, kernel_size=(5,5),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(128, kernel_size=(5,5),strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(256, kernel_size=(5,5),strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(512, kernel_size=(5,5),activation='relu')(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Flatten()(hl)
hl = Dense(1024, activation='relu')(hl)
hl = Dense(128, activation='relu')(hl)
ol = Dense(1, activation='sigmoid')(hl)

model = Model(il,ol)

model.summary()

model.compile(optimizer='adam', loss = binary_crossentropy, metrics=['accuracy'])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1024, 1024, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation                   │ (None, 1024, 1024, 3)       │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 1020, 1020, 8)       │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 509, 509, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 509, 509, 8)         │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 509, 509, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 505, 505, 32)        │           6,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 252, 252, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 252, 252, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 252, 252, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 83, 83, 128)         │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 41, 41, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 41, 41, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 41, 41, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 13, 13, 256)         │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 6, 6, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 2, 2, 512)           │       3,277,3

 Total params: 6,439,585 (24.57 MB)

 Trainable params: 6,437,713 (24.56 MB)

 Non-trainable params: 1,872 (7.31 KB)

#### 2) 학습

In [20]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    min_delta= 0,
    restore_best_weights= True
)

In [ ]:
hist = model.fit(train_x, train_y, epochs=10000, validation_split=.2,callbacks = [es])

Epoch 1/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 794ms/step - accuracy: 0.8764 - loss: 0.3181 - val_accuracy: 0.7857 - val_loss: 1.0068
Epoch 2/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 753ms/step - accuracy: 0.8268 - loss: 0.3682 - val_accuracy: 0.8367 - val_loss: 0.7633
Epoch 3/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 765ms/step - accuracy: 0.8598 - loss: 0.3681 - val_accuracy: 0.8673 - val_loss: 0.8285
Epoch 4/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 762ms/step - accuracy: 0.8322 - loss: 0.3609 - val_accuracy: 0.7959 - val_loss: 1.1740
Epoch 5/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 747ms/step - accuracy: 0.8527 - loss: 0.3575 - val_accuracy: 0.8061 - val_loss: 0.8681
Epoch 6/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 741ms/step - accuracy: 0.8413 - loss: 0.3473 - val_accuracy: 0.7959 - val_loss: 0.9639
Epoch 7/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 737ms/step - accuracy: 0.8667 - loss: 0.3194 - val_accuracy: 0.7857 - val_loss: 0.8738
Epoch 8/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 735ms/step - accuracy: 0.8471 -

#### 3) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
from sklearn.metrics import *
pred_val = model.predict(val_x)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
              precision    recall  f1-score   support

         0.0       0.95      0.75      0.84        28
         1.0       0.79      0.96      0.87        27

    accuracy                           0.85        55
   macro avg       0.87      0.86      0.85        55
weighted avg       0.87      0.85      0.85        55



In [ ]:
pred_val = model.predict(test_x)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
              precision    recall  f1-score   support

         0.0       0.85      0.73      0.79        30
         1.0       0.77      0.87      0.82        31

    accuracy                           0.80        61
   macro avg       0.81      0.80      0.80        61
weighted avg       0.81      0.80      0.80        61



## 모델2

In [ ]:
clear_session()

il = Input(shape=(1024,1024,3))
al = RandomTranslation(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3))(il)
hl = Conv2D(8,kernel_size=(5,5),padding='same',activation='relu')(al)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(32, kernel_size=(5,5),padding='same',activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(3,3))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(128, kernel_size=(5,5),padding='same',strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(256, kernel_size=(5,5),padding='same',strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(512, kernel_size=(5,5),padding='same',activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Flatten()(hl)
hl = Dense(1024, activation='relu')(hl)
hl = Dense(128, activation='relu')(hl)
ol = Dense(1, activation='sigmoid')(hl)

model = Model(il,ol)

model.summary()

model.compile(optimizer='adam', loss = binary_crossentropy, metrics=['accuracy'])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1024, 1024, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation                   │ (None, 1024, 1024, 3)       │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 1024, 1024, 8)       │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 511, 511, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 511, 511, 8)         │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 511, 511, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 511, 511, 32)        │           6,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 170, 170, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 170, 170, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 170, 170, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 57, 57, 128)         │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 10, 10, 256)         │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 4, 4, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 4, 4, 512)           │       3,277,3

 Total params: 6,439,585 (24.57 MB)

 Trainable params: 6,437,713 (24.56 MB)

 Non-trainable params: 1,872 (7.31 KB)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    min_delta= 0,
    restore_best_weights= True
)

In [ ]:
hist = model.fit(train_x, train_y, epochs=10000, validation_split=.2,callbacks = [es])

Epoch 1/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.6598 - loss: 0.9744 - val_accuracy: 0.4796 - val_loss: 2.6598
Epoch 2/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 710ms/step - accuracy: 0.6882 - loss: 0.9676 - val_accuracy: 0.5204 - val_loss: 7.2586
Epoch 3/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 710ms/step - accuracy: 0.7460 - loss: 0.7498 - val_accuracy: 0.5204 - val_loss: 4.3867
Epoch 4/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 713ms/step - accuracy: 0.7806 - loss: 0.5692 - val_accuracy: 0.5204 - val_loss: 1.8133
Epoch 5/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 716ms/step - accuracy: 0.7559 - loss: 0.5626 - val_accuracy: 0.5000 - val_loss: 0.8637
Epoch 6/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 715ms/step - accuracy: 0.7526 - loss: 0.5360 - val_accuracy: 0.7347 - val_loss: 0.8541
Epoch 7/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 715ms/step - accuracy: 0.7659 - loss: 0.5554 - val_accuracy: 0.7755 - val_loss: 0.9076
Epoch 8/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 723ms/step - accuracy: 0.8191 - loss: 0.4

In [ ]:
from sklearn.metrics import *
pred_val = model.predict(val_x)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
              precision    recall  f1-score   support

         0.0       0.63      0.93      0.75        28
         1.0       0.86      0.44      0.59        27

    accuracy                           0.69        55
   macro avg       0.75      0.69      0.67        55
weighted avg       0.74      0.69      0.67        55



In [ ]:
pred_val = model.predict(test_x)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
              precision    recall  f1-score   support

         0.0       0.63      0.73      0.68        30
         1.0       0.69      0.58      0.63        31

    accuracy                           0.66        61
   macro avg       0.66      0.66      0.65        61
weighted avg       0.66      0.66      0.65        61



## 모델3

In [ ]:
clear_session()

il = Input(shape=(1024,1024,3))
al = RandomTranslation(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3))(il)
hl = Conv2D(8,kernel_size=(5,5),activation='relu')(al)
hl = Conv2D(8,kernel_size=(5,5),activation='relu')(al)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(32, kernel_size=(5,5),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(128, kernel_size=(5,5),strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(256, kernel_size=(5,5),strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Flatten()(hl)
hl = Dense(1024, activation='relu')(hl)
hl = Dense(128, activation='relu')(hl)
ol = Dense(1, activation='sigmoid')(hl)

model = Model(il,ol)

model.summary()

model.compile(optimizer='adam', loss = binary_crossentropy, metrics=['accuracy'])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1024, 1024, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation                   │ (None, 1024, 1024, 3)       │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 1020, 1020, 8)       │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 509, 509, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 509, 509, 8)         │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 509, 509, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 505, 505, 32)        │           6,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 252, 252, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 252, 252, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 252, 252, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 83, 83, 128)         │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 41, 41, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 41, 41, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 41, 41, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 13, 13, 256)         │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 6, 6, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 9216)                │              

 Total params: 10,500,257 (40.06 MB)

 Trainable params: 10,499,409 (40.05 MB)

 Non-trainable params: 848 (3.31 KB)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    min_delta= 0,
    restore_best_weights= True
)

In [ ]:
hist = model.fit(train_x, train_y, epochs=10000, validation_split=.2,callbacks = [es])

Epoch 1/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 787ms/step - accuracy: 0.6225 - loss: 1.9075 - val_accuracy: 0.5204 - val_loss: 11.7573
Epoch 2/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 719ms/step - accuracy: 0.7093 - loss: 0.9316 - val_accuracy: 0.5000 - val_loss: 4.5246
Epoch 3/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 715ms/step - accuracy: 0.7092 - loss: 0.7518 - val_accuracy: 0.5204 - val_loss: 8.3953
Epoch 4/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 724ms/step - accuracy: 0.7721 - loss: 0.6794 - val_accuracy: 0.4898 - val_loss: 1.8703
Epoch 5/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 730ms/step - accuracy: 0.7757 - loss: 0.5141 - val_accuracy: 0.4286 - val_loss: 1.0638
Epoch 6/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 724ms/step - accuracy: 0.7707 - loss: 0.5055 - val_accuracy: 0.5306 - val_loss: 1.7574
Epoch 7/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 728ms/step - accuracy: 0.8308 - loss: 0.4376 - val_accuracy: 0.5510 - val_loss: 1.8921
Epoch 8/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 731ms/step - accuracy: 0.8477 - 

In [ ]:
from sklearn.metrics import *
pred_val = model.predict(val_x)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
              precision    recall  f1-score   support

         0.0       0.79      0.79      0.79        28
         1.0       0.78      0.78      0.78        27

    accuracy                           0.78        55
   macro avg       0.78      0.78      0.78        55
weighted avg       0.78      0.78      0.78        55



In [ ]:
pred_val = model.predict(test_x)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
              precision    recall  f1-score   support

         0.0       0.81      0.97      0.88        30
         1.0       0.96      0.77      0.86        31

    accuracy                           0.87        61
   macro avg       0.88      0.87      0.87        61
weighted avg       0.88      0.87      0.87        61



## 4.모델링 II
* **세부요구사항**
    - image_dataset_from_directory를 이용하여 training set, validation set을 만들어봅시다.
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Image Preprocessing Layer, Image Augmentation Layer를 통해 이미지를 변형시켜 봅시다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.

### (1) image_dataset_from_directory

### (2) Transfer Learning

#### 1) Inception V3 모델을 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 False로 설정하여 불러온 모델의 모든 레이어들을 Frozen 시켜 가중치를 고정합니다.

In [ ]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape

((489, 1024, 1024, 3),
 (489,),
 (55, 1024, 1024, 3),
 (55,),
 (61, 1024, 1024, 3),
 (61,))

# data resize

In [ ]:
from skimage.transform import resize

# 이미지 배열의 크기를 변경하기 위한 빈 배열 생성
train_x_299 = np.zeros((489, 299, 299, 3))
val_x_299 = np.zeros((55, 299, 299, 3))
test_x_299 = np.zeros((61, 299, 299, 3))

# 각 이미지를 299x299 크기로 리사이징
for i in range(train_x.shape[0]):
    train_x_299[i] = resize(train_x[i], (299, 299, 3), anti_aliasing=False)

for i in range(val_x.shape[0]):
    val_x_299[i] = resize(val_x[i], (299, 299, 3), anti_aliasing=False)

for i in range(test_x.shape[0]):
    test_x_299[i] = resize(test_x[i], (299, 299, 3), anti_aliasing=False)


In [54]:
train_x_299.shape, val_x_299.shape, test_x_299.shape

((489, 299, 299, 3), (55, 299, 299, 3), (61, 299, 299, 3))

In [21]:
from keras.layers import RandomContrast, RandomFlip

In [67]:
clear_session()


il = Input(shape=(299,299,3))

al = RandomTranslation(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3))(il)
al = RandomContrast(factor=(1.0, 1.5))(al)
al = RandomFlip(mode='horizontal_and_vertical')(al)


base_model = keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    )(al)

new_output = GlobalAveragePooling2D()(base_model)
hl = Dropout(.3)(new_output)
new_output = Dense(1024, activation = 'relu')(hl)
new_output = Dense(1, activation = 'sigmoid')(new_output)


model = Model(il, new_output)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 299, 299, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation                   │ (None, 299, 299, 3)         │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_contrast (RandomContrast)     │ (None, 299, 299, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_flip (RandomFlip)             │ (None, 299, 299, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_v3 (Functional)            │ (None, 8, 8, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │           1,025 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,901,985 (91.18 MB)

 Trainable params: 23,867,553 (91.05 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [68]:
model.compile(optimizer='adam', loss = binary_crossentropy, metrics=['accuracy'])

In [69]:
from keras.callbacks import EarlyStopping

In [70]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 20,
    min_delta= 0,
    restore_best_weights= True
)

In [71]:
hist = model.fit(train_x_299, train_y, epochs=10000, validation_data=(val_x_299, val_y),callbacks = [es])

Epoch 1/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 51s 877ms/step - accuracy: 0.7711 - loss: 0.5241 - val_accuracy: 0.5091 - val_loss: 47.6418
Epoch 2/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 621ms/step - accuracy: 0.8273 - loss: 0.4097 - val_accuracy: 0.5091 - val_loss: 11194.8203
Epoch 3/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 610ms/step - accuracy: 0.8653 - loss: 0.4048 - val_accuracy: 0.4909 - val_loss: 488.8303
Epoch 4/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 613ms/step - accuracy: 0.8935 - loss: 0.2882 - val_accuracy: 0.4909 - val_loss: 46.8634
Epoch 5/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 608ms/step - accuracy: 0.9116 - loss: 0.2387 - val_accuracy: 0.5091 - val_loss: 16.1239
Epoch 6/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 596ms/step - accuracy: 0.8748 - loss: 0.3702 - val_accuracy: 0.5818 - val_loss: 54.2661
Epoch 7/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 597ms/step - accuracy: 0.9311 - loss: 0.2247 - val_accuracy: 0.5636 - val_loss: 216.4101
Epoch 8/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 608ms/step - accura

In [72]:
from sklearn.metrics import *
pred_val = model.predict(val_x_299)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

1/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step
              precision    recall  f1-score   support

         0.0       0.82      1.00      0.90        27
         1.0       1.00      0.79      0.88        28

    accuracy                           0.89        55
   macro avg       0.91      0.89      0.89        55
weighted avg       0.91      0.89      0.89        55



In [73]:
pred_val = model.predict(test_x_299)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
              precision    recall  f1-score   support

         0.0       0.76      0.93      0.84        30
         1.0       0.92      0.71      0.80        31

    accuracy                           0.82        61
   macro avg       0.84      0.82      0.82        61
weighted avg       0.84      0.82      0.82        61



#### 2) Inception V3 모델과 연결하여 구조 설계
* Inception V3을 불러온 후, 다른 레이어와 연결하기
* Image Preprocessing Layer, Image Augmentation Layer를 사용해보세요.

In [ ]:
model.save('best.keras')

#### 3) 학습

#### 4) 성능 평가

# **Keras Tuner**

In [18]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.4 MB/s eta 0:00:00


In [19]:
import keras_tuner

In [30]:
def build_model(hp):

  il = Input(shape=(1024,1024,3))
  al = RandomTranslation(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3))(il)
  hl = Conv2D(8,kernel_size=(5,5),activation='relu')(al)
  hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
  hl = BatchNormalization()(hl)
  hl = Dropout(0.2)(hl)
  hl = Conv2D(32, kernel_size=(5,5),activation='relu')(hl)
  hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
  hl = BatchNormalization()(hl)
  hl = Dropout(0.2)(hl)
  hl = Conv2D(128, kernel_size=(5,5),strides=(3,3),activation='relu')(hl)
  hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
  hl = BatchNormalization()(hl)
  hl = Dropout(0.2)(hl)
  hl = Conv2D(256, kernel_size=(5,5),strides=(3,3),activation='relu')(hl)
  hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
  hl = BatchNormalization()(hl)
  hl = Dropout(0.2)(hl)
  hl = Conv2D(512, kernel_size=(5,5),activation='relu')(hl)
  hl = BatchNormalization()(hl)
  hl = Dropout(0.2)(hl)
  hl = Flatten()(hl)
  hl = Dense(1024, activation='relu')(hl)
  hl = Dense(128, activation='relu')(hl)
  ol = Dense(1, activation='sigmoid')(hl)

  model = Model(il,ol)

  model.summary()

  model.compile(optimizer='adam', loss = binary_crossentropy, metrics=['accuracy'])

  return model

In [31]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=7)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 1024, 1024, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation_1                 │ (None, 1024, 1024, 3)       │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 1020, 1020, 8)       │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 509, 509, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 509, 509, 8)         │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 509, 509, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 505, 505, 32)        │           6,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 252, 252, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 252, 252, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 252, 252, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 83, 83, 128)         │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 41, 41, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 41, 41, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 41, 41, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 13, 13, 256)         │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 6, 6, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 2, 2, 512)           │       3,277,3

 Total params: 6,439,585 (24.57 MB)

 Trainable params: 6,437,713 (24.56 MB)

 Non-trainable params: 1,872 (7.31 KB)

In [32]:
from keras.callbacks import EarlyStopping

In [33]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    min_delta= 0,
    restore_best_weights= True
)

In [35]:
tuner.search(train_x, train_y, epochs=1000, validation_data=(val_x, val_y),callbacks=[es])
best_model = tuner.get_best_models()[0]

Trial 1 Complete [00h 07m 36s]
val_loss: 0.36369267106056213

Best val_loss So Far: 0.36369267106056213
Total elapsed time: 00h 07m 36s


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1024, 1024, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation                   │ (None, 1024, 1024, 3)       │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 1020, 1020, 8)       │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 509, 509, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 509, 509, 8)         │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 509, 509, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 505, 505, 32)        │           6,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 252, 252, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 252, 252, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 252, 252, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 83, 83, 128)         │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 41, 41, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 41, 41, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 41, 41, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 13, 13, 256)         │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 6, 6, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 2, 2, 512)           │       3,277,3

 Total params: 6,439,585 (24.57 MB)

 Trainable params: 6,437,713 (24.56 MB)

 Non-trainable params: 1,872 (7.31 KB)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [37]:
from sklearn.metrics import *
pred_test = best_model.predict(test_x)
pred_test = np.where(pred_test>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
              precision    recall  f1-score   support

         0.0       0.91      0.70      0.79        30
         1.0       0.76      0.94      0.84        31

    accuracy                           0.82        61
   macro avg       0.84      0.82      0.82        61
weighted avg       0.84      0.82      0.82        61



In [22]:
from skimage.transform import resize

# 이미지 배열의 크기를 변경하기 위한 빈 배열 생성
train_x_128 = np.zeros((489, 128, 128, 3))
val_x_128 = np.zeros((55, 128, 128, 3))
test_x_128 = np.zeros((61, 128, 128, 3))

# 각 이미지를 299x299 크기로 리사이징
for i in range(train_x.shape[0]):
    train_x_128[i] = resize(train_x[i], (128, 128, 3), anti_aliasing=True)

for i in range(val_x.shape[0]):
    val_x_128[i] = resize(val_x[i], (128, 128, 3), anti_aliasing=True)

for i in range(test_x.shape[0]):
    test_x_128[i] = resize(test_x[i], (128, 128, 3), anti_aliasing=True)


KeyboardInterrupt: 

In [60]:
train_x_128.shape, val_x_128.shape, test_x_128.shape

((489, 128, 128, 3), (55, 128, 128, 3), (61, 128, 128, 3))

In [95]:
clear_session()

base_model = keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape= (128,128,3)
    )

new_output = GlobalAveragePooling2D()(base_model.output)
hl = Dropout(.3)(new_output)
hl = Flatten()(hl)
new_output = Dense(1024, activation = 'relu')(hl)
new_output = Dense(128, activation = 'relu')(hl)
new_output = Dense(1, activation = 'sigmoid')(new_output)


model = Model(base_model.inputs, new_output)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 63, 63, 32)     │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 63, 63, 32)     │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 63, 63, 32)     │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 61, 61, 32)     │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 61, 61, 32)     │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 61, 61, 32)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 61, 61, 64)     │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 61, 61, 64)     │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 61, 61, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 30, 30, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 30, 30, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 30, 30, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 30, 30, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 28, 28, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 28, 28, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 22,065,185 (84.17 MB)

 Trainable params: 22,030,753 (84.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [98]:
model.compile(optimizer='adam', loss = binary_crossentropy, metrics=['accuracy'])

In [99]:
from keras.callbacks import EarlyStopping

In [100]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 30,
    min_delta= 0,
    restore_best_weights= True
)

In [101]:
hist = model.fit(train_x_128, train_y, epochs=10000, validation_data=(val_x_128, val_y),callbacks = [es])

Epoch 1/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 110s 3s/step - accuracy: 0.7184 - loss: 0.5494 - val_accuracy: 0.6727 - val_loss: 3.7274
Epoch 2/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.9583 - loss: 0.1570 - val_accuracy: 0.5091 - val_loss: 438.1378
Epoch 3/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.9234 - loss: 0.2132 - val_accuracy: 0.5091 - val_loss: 628.6717
Epoch 4/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.9789 - loss: 0.1083 - val_accuracy: 0.4909 - val_loss: 167.4659
Epoch 5/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.8837 - loss: 0.2779 - val_accuracy: 0.5091 - val_loss: 1030.2540
Epoch 6/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.9455 - loss: 0.2165 - val_accuracy: 0.4909 - val_loss: 1996.5549
Epoch 7/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.9387 - loss: 0.1932 - val_accuracy: 0.4909 - val_loss: 3.0714
Epoch 8/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.9265 - los

In [102]:
from sklearn.metrics import *
pred_val = model.predict(val_x_128)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95        27
         1.0       0.96      0.93      0.95        28

    accuracy                           0.95        55
   macro avg       0.95      0.95      0.95        55
weighted avg       0.95      0.95      0.95        55



In [103]:
pred_val = model.predict(test_x_128)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93        30
         1.0       0.94      0.94      0.94        31

    accuracy                           0.93        61
   macro avg       0.93      0.93      0.93        61
weighted avg       0.93      0.93      0.93        61



In [22]:
from keras.models import Sequential
from keras.layers import Rescaling,RandomRotation,RandomFlip

In [49]:
clear_session()
base_model = keras.applications.InceptionV3(input_shape=(299, 299, 3),include_top=False, weights='imagenet')

model = Sequential([
  Rescaling(1./255),
  # 이미지 증강 레이어
  RandomRotation(factor=(-0.3, 0.3)),
  RandomFlip(mode='horizontal_and_vertical'),
  # InceptionV3 베이스 모델
  base_model,
  # 추가 레이어
  GlobalAveragePooling2D(),
  # Dense(512, activation='relu'),
  Dense(1, activation='sigmoid')])
  # 모델 컴파일
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
model.summary ()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_rotation (RandomRotation)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_flip (RandomFlip)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_v3 (Functional)            │ ?                           │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 21,768,352 (83.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [50]:
from keras.callbacks import EarlyStopping

In [51]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 50,
    min_delta= 0,
    restore_best_weights= True
)

In [52]:
hist = model.fit(train_x_299, train_y, epochs=10000, validation_data=(val_x_299, val_y),callbacks = [es])


Epoch 1/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 52s 764ms/step - accuracy: 0.7997 - loss: 0.4447 - val_accuracy: 0.5091 - val_loss: 1.8268
Epoch 2/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 572ms/step - accuracy: 0.8566 - loss: 0.3270 - val_accuracy: 0.4909 - val_loss: 7.0758
Epoch 3/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 585ms/step - accuracy: 0.8998 - loss: 0.2419 - val_accuracy: 0.5091 - val_loss: 1.0144
Epoch 4/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 580ms/step - accuracy: 0.9324 - loss: 0.1973 - val_accuracy: 0.4909 - val_loss: 3.2734
Epoch 5/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 593ms/step - accuracy: 0.9366 - loss: 0.1622 - val_accuracy: 0.4909 - val_loss: 0.9214
Epoch 6/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 588ms/step - accuracy: 0.9520 - loss: 0.1305 - val_accuracy: 0.4909 - val_loss: 4.5436
Epoch 7/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 585ms/step - accuracy: 0.9201 - loss: 0.2398 - val_accuracy: 0.4909 - val_loss: 4.5181
Epoch 8/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 584ms/step - accuracy: 0.9196 - loss:

In [53]:
from sklearn.metrics import *
pred_val = model.predict(val_x_299)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step
              precision    recall  f1-score   support

         0.0       0.96      0.86      0.91        28
         1.0       0.87      0.96      0.91        27

    accuracy                           0.91        55
   macro avg       0.91      0.91      0.91        55
weighted avg       0.91      0.91      0.91        55



In [54]:
pred_val = model.predict(test_x_299)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
              precision    recall  f1-score   support

         0.0       1.00      0.73      0.85        30
         1.0       0.79      1.00      0.89        31

    accuracy                           0.87        61
   macro avg       0.90      0.87      0.87        61
weighted avg       0.90      0.87      0.87        61



In [27]:
keras.backend.clear_session()

il = Input(shape=(299,299,3))

cl = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation = 'relu')(il)
pl = MaxPool2D(pool_size=(2,2) )(cl)
dr = Dropout(0.25)(pl)

cl = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation = 'relu')(dr)
cl = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation = 'relu')(cl)
pl = MaxPool2D(pool_size=(2,2) )(cl)
dr = Dropout(0.25)(pl)

cl = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation = 'relu')(dr)
cl = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation = 'relu')(cl)
pl = MaxPool2D(pool_size=(2,2) )(cl)
dr = Dropout(0.25)(pl)

cl = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation = 'relu')(dr)
cl = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation = 'relu')(cl)
pl = MaxPool2D(pool_size=(2,2) )(cl)
dr = Dropout(0.25)(pl)

fl = Flatten()(dr)
dl = Dense(1024, activation = 'relu')(fl)
dr = Dropout(0.25)(dl)
ol = Dense(1, activation='sigmoid')(dl)

model_mVGG = keras.models.Model(il,ol)

model_mVGG.compile(loss = 'binary_crossentropy', metrics=['accuracy'], optimizer='adam')

model_mVGG.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 299, 299, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 299, 299, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 149, 149, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 149, 149, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 149, 149, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 149, 149, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 74, 74, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 74, 74, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 74, 74, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 74, 74, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 37, 37, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 37, 37, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 37, 37, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 37, 37, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 18, 18, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 18, 18, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 165888)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │     169,870,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │           1,025 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 174,519,809 (665.74 MB)

 Trainable params: 174,519,809 (665.74 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
from keras.callbacks import EarlyStopping

In [30]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 30,
    min_delta= 0,
    restore_best_weights= True
)

In [32]:
hist = model_mVGG.fit(train_x_299, train_y, epochs=10000, validation_data=(val_x_299, val_y),callbacks = [es])


Epoch 1/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 185s 6s/step - accuracy: 0.4848 - loss: 3.1968 - val_accuracy: 0.5091 - val_loss: 0.6906
Epoch 2/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 510ms/step - accuracy: 0.5331 - loss: 0.6935 - val_accuracy: 0.5091 - val_loss: 0.6898
Epoch 3/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 503ms/step - accuracy: 0.5070 - loss: 0.6924 - val_accuracy: 0.6364 - val_loss: 0.6701
Epoch 4/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 453ms/step - accuracy: 0.6280 - loss: 0.6637 - val_accuracy: 0.6909 - val_loss: 0.7057
Epoch 5/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 506ms/step - accuracy: 0.6795 - loss: 0.6538 - val_accuracy: 0.7636 - val_loss: 0.4924
Epoch 6/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 453ms/step - accuracy: 0.7916 - loss: 0.4947 - val_accuracy: 0.8545 - val_loss: 0.5255
Epoch 7/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 504ms/step - accuracy: 0.8046 - loss: 0.4836 - val_accuracy: 0.8364 - val_loss: 0.3785
Epoch 8/10000
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 453ms/step - accuracy: 0.8364 - loss: 0.

In [33]:
from sklearn.metrics import *
pred_val = model_mVGG.predict(val_x_299)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 472ms/step
              precision    recall  f1-score   support

         0.0       0.85      0.81      0.83        27
         1.0       0.83      0.86      0.84        28

    accuracy                           0.84        55
   macro avg       0.84      0.84      0.84        55
weighted avg       0.84      0.84      0.84        55



In [34]:
pred_val = model_mVGG.predict(test_x_299)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step
              precision    recall  f1-score   support

         0.0       0.68      0.63      0.66        30
         1.0       0.67      0.71      0.69        31

    accuracy                           0.67        61
   macro avg       0.67      0.67      0.67        61
weighted avg       0.67      0.67      0.67        61



In [23]:

keras.backend.clear_session()

il = Input(shape=(1024,1024,3))
al = RandomTranslation(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3))(il)
al = RandomContrast(factor=(1.0, 1.5))(al)
al = RandomFlip(mode='horizontal_and_vertical')(al)

resnet = keras.applications.ResNet50(weights='imagenet',
                  include_top=False,
                  )(al)

pl = GlobalAveragePooling2D()(resnet)
ol = Dense(1, activation = 'sigmoid')(pl)

model_resnet = keras.models.Model(il, ol)

model_resnet.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [24]:
model_resnet.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1024, 1024, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation                   │ (None, 1024, 1024, 3)       │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_contrast (RandomContrast)     │ (None, 1024, 1024, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_flip (RandomFlip)             │ (None, 1024, 1024, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 32, 32, 2048)        │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │           2,049 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 23,536,641 (89.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [103]:
from keras.callbacks import EarlyStopping

In [25]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 20,
    min_delta= 0,
    restore_best_weights= True
)

In [29]:
hist = model_resnet.fit(train_x, train_y, epochs=10000, validation_data=(val_x, val_y),callbacks = [es], batch_size=8)


Epoch 1/10000
62/62 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.5823 - loss: 1.0592 - val_accuracy: 0.5091 - val_loss: 7561.3999
Epoch 2/10000
62/62 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.6279 - loss: 0.7669 - val_accuracy: 0.4909 - val_loss: 522.3011
Epoch 3/10000
19/62 ━━━━━━━━━━━━━━━━━━━━ 1:12 2s/step - accuracy: 0.7014 - loss: 0.6068

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import *
pred_val = model_resnet.predict(val_x_299)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

In [ ]:
pred_val = model_resnet.predict(test_x_299)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))